In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

In [2]:
data = pd.read_csv(r'C:\Users\harin\Downloads\air_quality_and _weather_prediction\Carbon_Emission.csv')

In [3]:
data.head()

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
0,overweight,female,pescatarian,daily,coal,public,NaN,often,230,frequently,210,large,4,7,26,1,No,['Metal'],"['Stove', 'Oven']",2238
1,obese,female,vegetarian,less frequently,natural gas,walk/bicycle,NaN,often,114,rarely,9,extra large,3,9,38,5,No,['Metal'],"['Stove', 'Microwave']",1892
2,overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,['Metal'],"['Oven', 'Microwave']",2595
3,overweight,male,omnivore,twice a day,wood,walk/bicycle,NaN,sometimes,157,rarely,74,medium,3,20,5,7,Sometimes,"['Paper', 'Plastic', 'Glass', 'Metal']","['Microwave', 'Grill', 'Airfryer']",1074
4,obese,female,vegetarian,daily,coal,private,diesel,often,266,very frequently,8457,large,1,3,5,6,Yes,['Paper'],['Oven'],4743


In [4]:
data.isnull().sum()

Body Type                           0
Sex                                 0
Diet                                0
How Often Shower                    0
Heating Energy Source               0
Transport                           0
Vehicle Type                     6721
Social Activity                     0
Monthly Grocery Bill                0
Frequency of Traveling by Air       0
Vehicle Monthly Distance Km         0
Waste Bag Size                      0
Waste Bag Weekly Count              0
How Long TV PC Daily Hour           0
How Many New Clothes Monthly        0
How Long Internet Daily Hour        0
Energy efficiency                   0
Recycling                           0
Cooking_With                        0
CarbonEmission                      0
dtype: int64

In [5]:
data.fillna(data.mean(numeric_only=True),inplace=True)

In [7]:
for col in data.select_dtypes(include='object').columns:
    data[col].fillna(data[col].mode()[0],inplace=True)

C:\Users\harin\AppData\Local\Temp\ipykernel_18532\2882979949.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0],inplace=True)


In [8]:
data = data.drop(['Recycling','Cooking_With'],axis=True)

In [9]:
data.head()

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,CarbonEmission
0,overweight,female,pescatarian,daily,coal,public,lpg,often,230,frequently,210,large,4,7,26,1,No,2238
1,obese,female,vegetarian,less frequently,natural gas,walk/bicycle,lpg,often,114,rarely,9,extra large,3,9,38,5,No,1892
2,overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,2595
3,overweight,male,omnivore,twice a day,wood,walk/bicycle,lpg,sometimes,157,rarely,74,medium,3,20,5,7,Sometimes,1074
4,obese,female,vegetarian,daily,coal,private,diesel,often,266,very frequently,8457,large,1,3,5,6,Yes,4743


In [11]:
data.isnull().sum()

Body Type                        0
Sex                              0
Diet                             0
How Often Shower                 0
Heating Energy Source            0
Transport                        0
Vehicle Type                     0
Social Activity                  0
Monthly Grocery Bill             0
Frequency of Traveling by Air    0
Vehicle Monthly Distance Km      0
Waste Bag Size                   0
Waste Bag Weekly Count           0
How Long TV PC Daily Hour        0
How Many New Clothes Monthly     0
How Long Internet Daily Hour     0
Energy efficiency                0
CarbonEmission                   0
dtype: int64

In [12]:
mappings={
    'Social Activity':{'often':3,'sometimes':2,'rarely':1,'never':0},
    'Frequency of Traveling by Air':{'very frequently':4,'frequently':3,'rarely':2,'never':1},
    'Waste Bag Size':{'small':1,'medium':2,'large':3,'extra large':4},
    'Energy efficiency':{'No':0,'Sometimes':1,'Yes':2},
    'How Often Shower':{'daily':5,'more frequently':15,'less frequently':2,'twice a day':10}
}

In [13]:
for column,mapping in mappings.items():
    if column in data.columns:
        data[column]=data[column].map(mapping)

In [35]:
from sklearn.preprocessing import LabelEncoder
categorical_columns = ['Body Type','Sex','Diet','Transport','Vehicle Type','Heating Energy Source','How Often Shower']
label_encoders={}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col]=le

In [36]:
print(data.head(5))

   Body Type  Sex  Diet  How Often Shower  Heating Energy Source  Transport  \
0          2    0     1                 3                      0          1   
1          1    0     3                 2                      2          2   
2          2    1     0                 1                      3          0   
3          2    1     0                 0                      3          2   
4          1    0     3                 3                      0          0   

   Vehicle Type  Social Activity  Monthly Grocery Bill  \
0             3                3              0.777025   
1             3                3             -0.828947   
2             4                0             -0.496677   
3             3                2             -0.233630   
4             0                3              1.275430   

   Frequency of Traveling by Air  Vehicle Monthly Distance Km  Waste Bag Size  \
0                              3                    -0.657677               3   
1             

In [37]:
non_numeric_cols =data.dtypes[data.dtypes.apply(lambda x:x not in ['int64','flost64'])].index
if non_numeric_cols.any():
    print("Non numeric columns after preprocessing:",non_numeric_cols)
    raise ValueError("Some columns still non_numeric. Please review it again")

Non numeric columns after preprocessing: Index(['Monthly Grocery Bill', 'Vehicle Monthly Distance Km',
       'Waste Bag Weekly Count', 'How Long TV PC Daily Hour',
       'How Many New Clothes Monthly', 'How Long Internet Daily Hour'],
      dtype='object')


ValueError: Some columns still non_numeric. Please review it again

In [18]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
numeric_cols =['Monthly Grocery Bill','Vehicle Monthly Distance Km','Waste Bag Weekly Count','How Long TV PC Daily Hour',
               'How Many New Clothes Monthly','How Long Internet Daily Hour']
scaler = StandardScaler()
data[numeric_cols]=scaler.fit_transform(data[numeric_cols])

In [19]:
print(data.head(5))

   Body Type  Sex  Diet  How Often Shower  Heating Energy Source  Transport  \
0          2    0     1                 5                      0          1   
1          1    0     3                 2                      2          2   
2          2    1     0                15                      3          0   
3          2    1     0                10                      3          2   
4          1    0     3                 5                      0          0   

   Vehicle Type  Social Activity  Monthly Grocery Bill  \
0             3                3              0.777025   
1             3                3             -0.828947   
2             4                0             -0.496677   
3             3                2             -0.233630   
4             0                3              1.275430   

   Frequency of Traveling by Air  Vehicle Monthly Distance Km  Waste Bag Size  \
0                              3                    -0.657677               3   
1             

In [20]:
X = data.drop('CarbonEmission',axis=1)
y = data['CarbonEmission']

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [23]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test,y_pred)
print(rmse)

91782.8796802


In [24]:
import joblib
#save the trained model and preprocessing objects
joblib.dump(model,"carbon_footprint_model.pkl")
joblib.dump(label_encoders,"label_encoders.pkl")
joblib.dump(scaler,"scaler.pkl")
print("Model and preprocessors saved successfully")

Model and preprocessors saved successfully


In [25]:
model  =joblib.load('carbon_footprint_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')
scaler  = joblib.load('scaler.pkl')

In [26]:
input_data = {
    'Body Type':['overweight'],
    'Sex':['female'],
    'Diet':['pescatarian'],
    'How Often Shower':['daily'],
    'Heating Energy Source':['coal'],
    'Transport':['public'],
    'Vehicle Type':['petrol'],
    'Social Activity':['often'],
    'Monthly Grocery Bill':[230],
    'Frequency of Traveling by Air':['frequently'],
    'Vehicle Monthly Distance Km':[210],
    'Waste Bag Size':['large'],
    'Waste Bag Weekly Count':[4],
    'How Long TV PC Daily Hour':[7],
    'How Many New Clothes Monthly':[26],
    'How Long Internet Daily Hour':[1],
    'Energy efficiency':['No']
}

In [27]:
#convert the input data to a Dataframe
input_df = pd.DataFrame(input_data)

In [28]:
for column,mapping in mappings.items():
    if column in input_df.columns:
        input_df[column]=input_df[column].map(mapping)

In [29]:
print(label_encoders)

{'Body Type': LabelEncoder(), 'Sex': LabelEncoder(), 'Diet': LabelEncoder(), 'Transport': LabelEncoder(), 'Vehicle Type': LabelEncoder(), 'Heating Energy Source': LabelEncoder()}


In [30]:
for col, le in label_encoders.items():
    input_df[col] = le.transform(input_df[col].astype(str))

In [31]:
numerical_columns =['Monthly Grocery Bill','Vehicle Monthly Distance Km','Waste Bag Weekly Count','How Long TV PC Daily Hour','How Many New Clothes Monthly','How Long Internet Daily Hour']
input_df[numerical_columns]=scaler.transform(input_df[numerical_columns])

In [32]:
predicted_emission = model.predict(input_df)

In [33]:
print(f"Predicted Carbon Emission:{predicted_emission[0]}")

Predicted Carbon Emission:2401.11
